## Import library

In [ ]:
from tqdm.notebook import trange,tqdm
import numpy as np

## Configure the device

Device: ChipWhisperer-Lite

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C' 
SS_VER='SS_VER_2_1'

import chipwhisperer as cw

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 SS_VER=$3 -j

In [ ]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

## Use our collected trace

If you do not have a ChipWhisperer device, you can use the traces we collected and skip the following trace collecting part. It may take some time to load the trace.

Remember to replace the file path.

In [ ]:
your_path = "add_your_path_here"
trace_profile = np.loadtxt(your_path+"/data/trace_profile.txt",delimiter=',')
trace_garb = np.loadtxt(your_path+"/data/trace_garb.txt",delimiter=',')
trace_eval = np.loadtxt(your_path+"/data/trace_eval.txt",delimiter=',')

## Collecting Data

In [ ]:
# generate keys and plaintexts
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []
text = []
text_random = []

n_iter = 2000 # number of test traces
n_profile = 200

for i in range(4*n_iter):
    key, text_t = ktp.next()
    text.append(text_t)
    

for i in range(n_profile):
    _, text_t = ktp.next()
    text_random.append(text_t)

In [ ]:
def capture(key, text):
    target.set_key(key)
    
    scope.arm()
    
    target.simpleserial_write('p', text)
    
    ret = scope.capture()
    if ret:
        print("Target timed out!")
    
    response = target.simpleserial_read('r', 16)
    
    return scope.get_last_trace()

In [ ]:
trace_garb = []
trace_eval = []
trace_profile = []

pos_begin = 150
pos_end = 7150


scope.adc.samples = 8000
scope.adc.decimate = 4

for t in tqdm(text_random):
    trace_profile.append(np.array(capture(key, t))[pos_begin:pos_end])
    


for i in trange(4*n_iter):
    trace_garb.append(np.array(capture(key, text[i]))[pos_begin:pos_end])

    
for i in trange(4*n_iter):
    trace_eval.append(np.array(capture(key, text[i]))[pos_begin:pos_end])

In [ ]:
# for cw lite
np.savetxt("D:\\prj\\chipwhisperer\\jupyter\\user\\open_book\\data\\trace_profile.txt",trace_profile,fmt='%f',delimiter=',')
np.savetxt("D:\\prj\\chipwhisperer\\jupyter\\user\\open_book\\data\\trace_garb.txt",trace_garb,fmt='%f',delimiter=',')
np.savetxt("D:\\prj\\chipwhisperer\\jupyter\\user\\open_book\\data\\trace_eval.txt",trace_eval,fmt='%f',delimiter=',')

## Data Processing

In [ ]:
# select point of interest (POI) by variance
train_cwnano_var = []
poi = []

n_poi = 1
interval = 500
trace_len = pos_end-pos_begin
pieces = int(trace_len/interval)

trace_garb = np.array(trace_garb)

for i in range(trace_len):
    train_cwnano_var.append(np.var(trace_garb[:,i]))

for i in range(pieces):
    poi_nano_idx = np.array(train_cwnano_var[i*interval:i*interval+interval]).argsort()[-n_poi:][::-1]

    for idx in poi_nano_idx:
        true_idx = i*interval+idx
        poi.append(true_idx)

In [ ]:
def mean(X):
    return np.sum(X, axis=0)/len(X)

def std_dev(X):
    X_bar = mean(X)
    return np.sqrt(np.sum((X-X_bar)**2, axis=0))

def corr(X, Y):
    X_bar = mean(X)
    Y_bar = mean(Y)
    return np.sum((X-X_bar)*(Y-Y_bar), axis=0)/(std_dev(X)*std_dev(Y))

In [ ]:
cnt = 0
sig = 0

trace_avg = np.mean(np.array(trace_profile), axis=0)

for i in trange(n_iter):
    
    # victim trace from lite
    v_trace = trace_garb[i*4:i*4+4]
    # template trace from nano
    t_trace = trace_eval[i*4]
    
    trace_compact = []

    for ii in range(4):
        trace_compact.append(np.array([(v_trace[ii][p])-trace_avg[p] for p in poi]))

    trace_ref_compact = np.array([t_trace[p]-trace_avg[p] for p in poi])
    
    c = np.zeros(4)

    for ii in range(4):
        c[ii] = corr(trace_compact[ii],trace_ref_compact)
        
    sig += c[0] - max(c[1] , c[2] , c[3])
        
    if c.argmax(axis=0) == 0:
        cnt += 1
        
print("Accuracy:\t", cnt/n_iter)        
print("Significance:\t", sig/n_iter)

In [ ]:
scope.dis()